<a href="https://colab.research.google.com/github/Dee-Nwanjah/SQL-Database-Fundamental-Projects/blob/main/3.)Inventory_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# SQL SETUP

# Step 1: Installing and importing required packages
!pip install ipython-sql pandas numpy

import pandas as pd
import numpy as np
import sqlite3
from IPython import get_ipython

# Step 2: Creating database connection
conn = sqlite3.connect('customer_database.db')  # Creates a file-based database
print("✅ Database connection created successfully!")

# Step 3: Loading SQL magic
get_ipython().run_line_magic('load_ext', 'sql')
get_ipython().run_line_magic('sql', 'sqlite:///customer_database.db')
print("✅ SQL magic loaded successfully!")

# CREATING A SAMPLE DATA
print("📊 Creating sample data...")

# Creating sample datasets
np.random.seed(42)

# Products data
products_data = {
    'product_id': range(1, 51),  # Smaller dataset for learning
    'product_name': [f'Product_{i}' for i in range(1, 51)],
    'category': np.random.choice(['Electronics', 'Clothing', 'Books', 'Home', 'Sports'], 50),
    'price': np.round(np.random.uniform(10, 200, 50), 2),
    'stock_quantity': np.random.randint(0, 100, 50),
    'reorder_level': np.random.randint(5, 20, 50)
}

# Orders data
orders_data = {
    'order_id': range(1, 501),  # Smaller dataset for learning
    'customer_id': np.random.randint(1, 101, 500),
    'product_id': np.random.randint(1, 51, 500),
    'order_date': pd.date_range('2024-01-01', periods=500, freq='h')[:500].strftime('%Y-%m-%d'),
    'quantity': np.random.randint(1, 5, 500),
    'total_amount': np.round(np.random.uniform(20, 300, 500), 2)
}

# Customers data
customers_data = {
    'customer_id': range(1, 101),
    'first_name': [f'Customer_{i}' for i in range(1, 101)],
    'last_name': [f'LastName_{i}' for i in range(1, 101)],
    'city': np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston'], 100),
    'age': np.random.randint(18, 65, 100)
}

# Converting to DataFrames
products_df = pd.DataFrame(products_data)
orders_df = pd.DataFrame(orders_data)
customers_df = pd.DataFrame(customers_data)

# Saving it to the database using pandas
products_df.to_sql('products', conn, if_exists='replace', index=False)
orders_df.to_sql('orders', conn, if_exists='replace', index=False)
customers_df.to_sql('customers', conn, if_exists='replace', index=False)

print("✅ Sample data created successfully!")
print(f"   - Products: {len(products_df)} records")
print(f"   - Orders: {len(orders_df)} records")
print(f"   - Customers: {len(customers_df)} records")

# =============================================================================
# TESTING THE SETUP
# =============================================================================

print("\n🧪 Testing the setup...")

# Test 1: Check if tables were created
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
print(f"✅ Tables created: {[table[0] for table in tables]}")

# Test 2: Check if I can query data
test_query = "SELECT COUNT(*) as total_products FROM products"
result = pd.read_sql(test_query, conn)
print(f"✅ Test query successful: {result.iloc[0]['total_products']} products found")

print("\n🎉 Setup complete! You can now run SQL queries.")
print("\n" + "="*50)

✅ Database connection created successfully!
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
✅ SQL magic loaded successfully!
📊 Creating sample data...
✅ Sample data created successfully!
   - Products: 50 records
   - Orders: 500 records
   - Customers: 100 records

🧪 Testing the setup...
✅ Tables created: ['products', 'orders', 'customers']
✅ Test query successful: 50 products found

🎉 Setup complete! You can now run SQL queries.



Inventory Tracking

In [10]:
# Basic Inventory Check

# Define the SQL query
inventory_check_query = """
SELECT
    product_id,
    product_name,
    stock_quantity,
    reorder_level,
    CASE
        WHEN stock_quantity = 0 THEN 'OUT_OF_STOCK'
        WHEN stock_quantity <= reorder_level THEN 'LOW_STOCK'
        ELSE 'ADEQUATE'
    END as stock_status
FROM products
ORDER BY stock_quantity ASC
LIMIT 10;
"""

# Execute the SQL query using pandas read_sql
inventory_check_df = pd.read_sql(inventory_check_query, conn)

# Display the results
display(inventory_check_df)

,product_id,product_name,stock_quantity,reorder_level,stock_status
0,7,Product_7,0,18,OUT_OF_STOCK
1,6,Product_6,2,15,LOW_STOCK
2,23,Product_23,3,16,LOW_STOCK
3,8,Product_8,4,13,LOW_STOCK
4,41,Product_41,5,5,LOW_STOCK
5,12,Product_12,8,7,ADEQUATE
6,30,Product_30,12,8,ADEQUATE
7,10,Product_10,13,17,LOW_STOCK
8,14,Product_14,14,10,ADEQUATE
9,26,Product_26,14,15,LOW_STOCK


In [15]:
# Products that need reordering

# Define the SQL query
reorder_query = """
SELECT
    product_name,
    category,
    stock_quantity,
    reorder_level,
    (reorder_level - stock_quantity) as units_to_order
FROM products
WHERE stock_quantity <= reorder_level
ORDER BY (reorder_level - stock_quantity) DESC;
"""

# Execute the SQL query using pandas read_sql
reorder_df = pd.read_sql(reorder_query, conn)

# Display the results
display(reorder_df)

,product_name,category,stock_quantity,reorder_level,units_to_order
0,Product_7,Books,0,18,18
1,Product_6,Clothing,2,15,13
2,Product_23,Home,3,16,13
3,Product_8,Books,4,13,9
4,Product_10,Sports,13,17,4
5,Product_26,Books,14,15,1
6,Product_41,Home,5,5,0


In [27]:
# Sales velocity (how fast products sell)

# Define the SQL query
sales_velocity_query = """
SELECT
    p.product_name,
    p.stock_quantity,
    COUNT(o.order_id) as total_orders,
    SUM(o.quantity) as total_sold,
    ROUND(CAST(SUM(o.quantity) AS REAL) / 30.0, 2) as avg_daily_sales
FROM products p
LEFT JOIN orders o ON p.product_id = o.product_id
GROUP BY p.product_id, p.product_name, p.stock_quantity
ORDER BY avg_daily_sales DESC
LIMIT 10;
"""

# Execute the SQL query using pandas read_sql
sales_velocity_df = pd.read_sql(sales_velocity_query, conn)

# Display the results
display(sales_velocity_df)

,product_name,stock_quantity,total_orders,total_sold,avg_daily_sales
0,Product_47,61,20,54,1.80
1,Product_26,14,18,40,1.33
2,Product_33,58,15,40,1.33
3,Product_5,77,16,39,1.30
4,Product_38,44,13,37,1.23
5,Product_37,41,14,36,1.20
6,Product_14,14,13,35,1.17
7,Product_8,4,13,34,1.13
8,Product_11,26,14,33,1.10
9,Product_15,89,12,33,1.10


In [19]:
# Inventory value by category

# Define the SQL query
inventory_value_query = """
SELECT
    category,
    COUNT(*) as product_count,
    SUM(stock_quantity) as total_inventory,
    ROUND(SUM(stock_quantity * price), 2) as inventory_value,
    ROUND(AVG(price), 2) as avg_price
FROM products
GROUP BY category
ORDER BY inventory_value DESC;
"""

# Execute the SQL query using pandas read_sql
inventory_value_df = pd.read_sql(inventory_value_query, conn)

# Display the results
display(inventory_value_df)

,category,product_count,total_inventory,inventory_value,avg_price
0,Sports,10,606,73953.43,115.76
1,Home,13,618,59455.25,101.42
2,Clothing,10,470,57054.43,118.98
3,Electronics,7,379,40997.89,115.37
4,Books,10,354,35826.76,92.03


In [21]:
# Products with no recent sales (slow moving)

# Define the SQL query
slow_moving_query = """
SELECT
    p.product_name,
    p.category,
    p.stock_quantity,
    p.price,
    MAX(o.order_date) as last_sale_date,
    CASE
        WHEN MAX(o.order_date) IS NULL THEN 'NEVER_SOLD'
        WHEN MAX(o.order_date) < date('now', '-30 days') THEN 'SLOW_MOVING'
        ELSE 'ACTIVE'
    END as product_status
FROM products p
LEFT JOIN orders o ON p.product_id = o.product_id
GROUP BY p.product_id, p.product_name, p.category, p.stock_quantity, p.price
HAVING product_status IN ('NEVER_SOLD', 'SLOW_MOVING')
ORDER BY last_sale_date ASC;
"""

# Execute the SQL query using pandas read_sql
slow_moving_df = pd.read_sql(slow_moving_query, conn)

# Display the results
display(slow_moving_df)

,product_name,category,stock_quantity,price,last_sale_date,product_status
0,Product_16,Clothing,41,186.07,2024-01-08,SLOW_MOVING
1,Product_48,Sports,74,136.53,2024-01-13,SLOW_MOVING
2,Product_28,Clothing,28,62.44,2024-01-14,SLOW_MOVING
3,Product_9,Books,89,90.78,2024-01-16,SLOW_MOVING
4,Product_34,Electronics,85,65.76,2024-01-16,SLOW_MOVING
5,Product_44,Clothing,43,95.40,2024-01-16,SLOW_MOVING
6,Product_49,Clothing,91,122.35,2024-01-16,SLOW_MOVING
7,Product_50,Home,88,62.20,2024-01-16,SLOW_MOVING
8,Product_1,Home,34,55.79,2024-01-17,SLOW_MOVING
9,Product_19,Electronics,62,118.38,2024-01-17,SLOW_MOVING


In [26]:
# Daily sales trend

# Define the SQL query
daily_sales_query = """
SELECT
    order_date,
    COUNT(order_id) as daily_orders,
    SUM(quantity) as daily_units_sold,
    ROUND(SUM(total_amount), 2) as daily_revenue
FROM orders
GROUP BY order_date
ORDER BY order_date DESC;
"""

# Execute the SQL query using pandas read_sql
daily_sales_df = pd.read_sql(daily_sales_query, conn)

# Display the results
display(daily_sales_df)

,order_date,daily_orders,daily_units_sold,daily_revenue
0,2024-01-21,20,50,3631.62
1,2024-01-20,24,64,3912.17
2,2024-01-19,24,56,4148.20
3,2024-01-18,24,61,4507.46
4,2024-01-17,24,59,3581.52
5,2024-01-16,24,51,3355.39
6,2024-01-15,24,64,3600.79
7,2024-01-14,24,52,3744.17
8,2024-01-13,24,53,3796.26
9,2024-01-12,24,50,3730.19


In [30]:
# Top selling products this month

# Define the SQL query
top_selling_query = """
SELECT
    p.product_name,
    p.category,
    p.price,
    COUNT(o.order_id) as order_count,
    SUM(o.quantity) as total_quantity_sold,
    ROUND(SUM(o.total_amount), 2) as total_revenue
FROM products p
JOIN orders o ON p.product_id = o.product_id
GROUP BY p.product_id, p.product_name, p.category, p.price
ORDER BY total_quantity_sold DESC
LIMIT 10;
"""

# Execute the SQL query using pandas read_sql
top_selling_df = pd.read_sql(top_selling_query, conn)

# Display the results
display(top_selling_df)

,product_name,category,price,order_count,total_quantity_sold,total_revenue
0,Product_47,Clothing,137.08,20,54,3615.21
1,Product_26,Books,193.40,18,40,2592.32
2,Product_33,Home,85.03,15,40,2390.13
3,Product_5,Sports,42.94,16,39,2229.68
4,Product_38,Sports,160.13,13,37,2280.93
5,Product_37,Books,145.15,14,36,2444.32
6,Product_14,Clothing,95.45,13,35,2420.84
7,Product_8,Books,153.52,13,34,2274.96
8,Product_11,Home,117.86,14,33,2117.04
9,Product_15,Home,85.08,12,33,1587.26


In [31]:
# PYTHON CELL: Combining SQL with Python for analysis
# Getting low stock products and analyze them
low_stock_query = """
SELECT
    product_name,
    category,
    stock_quantity,
    reorder_level,
    price
FROM products
WHERE stock_quantity <= reorder_level
"""

# Executing query to get results as DataFrame
low_stock_df = pd.read_sql(low_stock_query, conn)

# Analyzing with Python
print("📊 LOW STOCK ANALYSIS")
print("=" * 40)
print(f"Total products needing reorder: {len(low_stock_df)}")
print(f"Categories affected: {low_stock_df['category'].nunique()}")
print(f"Total value of low stock items: ${(low_stock_df['stock_quantity'] * low_stock_df['price']).sum():.2f}")

print("\nLow stock by category:")
category_analysis = low_stock_df.groupby('category').agg({
    'product_name': 'count',
    'stock_quantity': 'sum'
}).round(2)
print(category_analysis)


📊 LOW STOCK ANALYSIS
Total products needing reorder: 7
Categories affected: 4
Total value of low stock items: $5258.38

Low stock by category:
          product_name  stock_quantity
category                              
Books                3              18
Clothing             1               2
Home                 2               8
Sports               1              13


In [32]:
# PYTHON CELL: Creating a simple inventory dashboard

def create_inventory_summary():
    """Create a simple text-based inventory dashboard"""

    # Getting key metrics
    total_products = pd.read_sql("SELECT COUNT(*) as count FROM products", conn).iloc[0]['count']

    out_of_stock = pd.read_sql("""
        SELECT COUNT(*) as count
        FROM products
        WHERE stock_quantity = 0
    """, conn).iloc[0]['count']

    low_stock = pd.read_sql("""
        SELECT COUNT(*) as count
        FROM products
        WHERE stock_quantity > 0 AND stock_quantity <= reorder_level
    """, conn).iloc[0]['count']

    total_inventory_value = pd.read_sql("""
        SELECT ROUND(SUM(stock_quantity * price), 2) as value
        FROM products
    """, conn).iloc[0]['value']

    # Print dashboard
    print("🏢 INVENTORY DASHBOARD")
    print("=" * 50)
    print(f"📦 Total Products: {total_products}")
    print(f"❌ Out of Stock: {out_of_stock}")
    print(f"⚠️  Low Stock: {low_stock}")
    print(f"✅ Adequate Stock: {total_products - out_of_stock - low_stock}")
    print(f"💰 Total Inventory Value: ${total_inventory_value:,}")
    print("=" * 50)

    # Stock status percentages
    print(f"Out of Stock: {(out_of_stock/total_products)*100:.1f}%")
    print(f"Low Stock: {(low_stock/total_products)*100:.1f}%")
    print(f"Adequate: {((total_products-out_of_stock-low_stock)/total_products)*100:.1f}%")

# Run the dashboard
create_inventory_summary()

🏢 INVENTORY DASHBOARD
📦 Total Products: 50
❌ Out of Stock: 1
⚠️  Low Stock: 6
✅ Adequate Stock: 43
💰 Total Inventory Value: $267,287.76
Out of Stock: 2.0%
Low Stock: 12.0%
Adequate: 86.0%
